In [344]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')  # Ignore all warnings

In [345]:
# Load the data into a Pandas DataFrame
market_data_df = pd.read_csv(
    "data/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
market_data_df.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [346]:
# Generate summary statistics
market_data_df.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [347]:
# Plot your data to see what's in your DataFrame
market_data_df.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [348]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_data_scaled = StandardScaler().fit_transform(market_data_df[["price_change_percentage_24h", 
                                                                    "price_change_percentage_7d", 
                                                                    "price_change_percentage_14d",
                                                                    "price_change_percentage_30d",
                                                                    "price_change_percentage_60d",
                                                                    "price_change_percentage_200d",
                                                                    "price_change_percentage_1y"
                                                                    ]])
market_data_scaled[0:5]

array([[ 0.50852937,  0.49319307,  0.77220043,  0.23545963, -0.0674951 ,
        -0.35595348, -0.25163688],
       [ 0.18544589,  0.93444504,  0.55869212, -0.05434093, -0.27348273,
        -0.11575947, -0.19935211],
       [ 0.02177396, -0.70633685, -0.02168042, -0.06103015,  0.00800452,
        -0.55024692, -0.28206051],
       [-0.04076438, -0.81092807,  0.24945797, -0.05038797, -0.37316402,
        -0.45825882, -0.29554614],
       [ 1.19303608,  2.00095907,  1.76061001,  0.54584206, -0.29120287,
        -0.49984776, -0.27031695]])

In [349]:
# Create a DataFrame with the scaled data
market_data_scaled_df = pd.DataFrame(market_data_scaled, columns=["price_change_percentage_24h", 
                                                                    "price_change_percentage_7d", 
                                                                    "price_change_percentage_14d",
                                                                    "price_change_percentage_30d",
                                                                    "price_change_percentage_60d",
                                                                    "price_change_percentage_200d",
                                                                    "price_change_percentage_1y"
                                                                    ])
# Copy the crypto names from the original data
market_data_scaled_df["coin_id"] = market_data_df.index

# Set the coinid column as index
market_data_scaled_df = market_data_scaled_df.set_index("coin_id")

# Display sample data
market_data_scaled_df.head(5)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [350]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,11))


In [351]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(market_data_scaled_df)
    inertia.append(k_model.inertia_)

In [352]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)

In [353]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_orig_plot = elbow_df.hvplot.line(
    x="k", 
    y="inertia", 
    title="Original Elbow Curve", 
    xticks=k,
    width=600,
    height=400
)
elbow_orig_plot


:Curve   [k]   (inertia)

In [354]:
# # Based on the elbow curve, 4 looks like the optimum number of clusters.  But I wanted to do look at the silhouette score of the k-values around what the elbow chart shows
# from sklearn.metrics import  silhouette_score

# print("=== Silhouette Score ===")

# for x in range(2, 7):
#     model = KMeans(n_clusters=x, random_state=1)
#     model.fit(market_data_scaled_df)
#     score = silhouette_score(market_data_scaled_df, model.labels_)
#     print(f"k{x} = {score}")


#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:**  Based on the visual representation of the elbow curve, 4 seems like the best k-value.  However, I wanted to do a silhouette score on the numbers around 4 to see how they vary.  Based on the silhouette score, 3 seems to be the optimal number.

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [355]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [356]:
# Fit the K-Means model using the scaled data
model.fit(market_data_scaled_df)

KMeans(n_clusters=4, random_state=1)

In [357]:
# Predict the clusters to group the cryptocurrencies using the scaled data
predictions = model.predict(market_data_scaled_df)

# Print the resulting array of cluster values.

predictions

array([3, 3, 1, 1, 3, 3, 3, 3, 3, 1, 1, 1, 1, 3, 1, 3, 1, 1, 3, 1, 1, 3,
       1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 0, 3, 1, 1, 2, 1, 1, 1, 1])

In [358]:
# Create a copy of the DataFrame
market_data_scaled_predictions_df = market_data_scaled_df.copy()


In [359]:
# Add a new column to the DataFrame with the predicted clusters
market_data_scaled_predictions_df["cluster"] = predictions

# Display sample data
market_data_scaled_predictions_df.head(5)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,cluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,3
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,3
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,1
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,1
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,3


In [360]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

cluster_orig_plot = market_data_scaled_predictions_df.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="cluster",
    hover_cols="coin_id",
    width=600,
    height=400
)
cluster_orig_plot

:NdOverlay   [cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [361]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [362]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.

market_data_pca = pca.fit_transform(market_data_scaled_df)

# View the first five rows of the DataFrame. 
market_data_pca[0:5]


array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [363]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_


array([0.3719856 , 0.34700813, 0.17603793])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total of the 3 principle components makes up approximately 89.4% of the data.

In [364]:
# Create a new DataFrame with the PCA data.
market_data_pca_df = pd.DataFrame(market_data_pca, columns = ["pca1", "pca2", "pca3"])


# Copy the crypto names from the original data
market_data_pca_df["coin_id"] = market_data_df.index


# Set the coinid column as index
market_data_pca_df = market_data_pca_df.set_index("coin_id")


# Display sample data
market_data_pca_df.head(5)


,pca1,pca2,pca3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In [365]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,11))

In [366]:
# Create an empty list to store the inertia values
inertia_pca = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(market_data_pca_df)
    inertia_pca.append(k_model.inertia_)



In [367]:
# Create a dictionary with the data to plot the Elbow curve
elbow_pca_data = {"k": k, "pca_inertia": inertia_pca}

# Create a DataFrame with the data to plot the Elbow curve
elbow_pca_df = pd.DataFrame(elbow_pca_data)

In [368]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_pca_plot = elbow_pca_df.hvplot.line(
    x="k", 
    y="pca_inertia", 
    title="PCA Elbow Curve", 
    xticks=k,
    width=600,
    height=400
)

elbow_pca_plot

:Curve   [k]   (pca_inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:**  Based on the elbow curve, 4 seems to be the most appropriate choice as  the curve starts to flatten out at this point.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** It seems to match thr result of k-value in the original data.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [369]:
# Initialize the K-Means model using the best value for k
pca_model = KMeans(n_clusters=4, random_state=1)


In [370]:
# Fit the K-Means model using the PCA data
pca_model.fit(market_data_pca_df)

KMeans(n_clusters=4, random_state=1)

In [371]:
# Predict the clusters to group the cryptocurrencies using the PCA data
pca_predictions = pca_model.predict(market_data_pca_df)

# Print the resulting array of cluster values.
pca_predictions

array([1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 3, 0, 0, 0, 0])

In [372]:
# Create a copy of the DataFrame with the PCA data
market_data_pca_predictions_df = market_data_pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
market_data_pca_predictions_df["cluster"] = pca_predictions

# Display sample data
market_data_pca_predictions_df.head(5)

,pca1,pca2,pca3,cluster
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,1
ethereum,-0.458261,0.458466,0.952877,1
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,1


In [373]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

cluster_pca_plot = market_data_pca_predictions_df.hvplot.scatter(
    x="pca1",
    y="pca2",
    by="cluster",
    hover_cols="coin_id",
    width=600,
    height=400
)
cluster_pca_plot


:NdOverlay   [cluster]
   :Scatter   [pca1]   (pca2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [376]:
# Composite plot to contrast the Elbow curves

elbow_orig_plot + elbow_pca_plot

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (pca_inertia)

In [375]:
# Composite plot to contrast the clusters
cluster_orig_plot + cluster_pca_plot

:Layout
   .NdOverlay.I  :NdOverlay   [cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [cluster]
      :Scatter   [pca1]   (pca2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Using the PCA to keep only the most impactful information reduces the noise in the data.  As a result, clearer clusters can be observed, as well as outliers.  This makes it easier to interpret.